<a href="https://colab.research.google.com/github/tlsehdgns1999/tlsehdgns1999/blob/main/%EC%97%98%EC%A7%80%EB%A1%9C%EC%BB%AC%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import random
import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, ReLU
from tensorflow.keras.models import Model

In [51]:
random.seed(1234)
np.random.seed(1234)
tf.random.set_seed(1234)

In [52]:
CFG = {
    'TRAIN_WINDOW_SIZE': 7,
    'PREDICT_SIZE': 21,
    'EPOCHS': 1,
    'LEARNING_RATE': 1e-4,
    'BATCH_SIZE': 4096,
    'SEED': 1234
}

In [53]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

In [54]:
seed_everything(CFG['SEED'])

In [55]:
train_data = pd.read_csv('./Desktop/open/train.csv').drop(columns=['ID', '제품'])

In [56]:
scale_max_dict = {}
scale_min_dict = {}

In [57]:
def scale_data(data):
    max_values = data.iloc[:, 4:].max(axis=1)
    min_values = data.iloc[:, 4:].min(axis=1)
    is_equal = max_values == min_values

    data.iloc[:, 4:] = np.where(is_equal[:, np.newaxis], 0, (data.iloc[:, 4:] - min_values[:, np.newaxis]) / (max_values - min_values)[:, np.newaxis])
    return max_values, min_values

scale_max_dict, scale_min_dict = scale_data(train_data)

C:\Users\butterflywings02\AppData\Local\Temp\ipykernel_24932\3225025377.py:6: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  data.iloc[:, 4:] = np.where(is_equal[:, np.newaxis], 0, (data.iloc[:, 4:] - min_values[:, np.newaxis]) / (max_values - min_values)[:, np.newaxis])


In [58]:
label_encoder = LabelEncoder()
categorical_columns = ['대분류', '중분류', '소분류', '브랜드']

for col in categorical_columns:
    label_encoder.fit(train_data[col])
    train_data[col] = label_encoder.transform(train_data[col])

class CustomDataset:
    def __init__(self, data, train_size=CFG['TRAIN_WINDOW_SIZE'], predict_size=CFG['PREDICT_SIZE'], is_inference=False, batch_size=CFG['BATCH_SIZE']):
        if not is_inference:
            max_values, min_values = scale_data(data)
            self.scale_max_dict = {col: max_values[i] for i, col in enumerate(data.columns[4:])}
            self.scale_min_dict = {col: min_values[i] for i, col in enumerate(data.columns[4:])}
            for col in categorical_columns:
                label_encoder.fit(data[col])
                data[col] = label_encoder.transform(data[col])
        self.data = data.values
        self.train_size = train_size
        self.predict_size = predict_size
        self.window_size = self.train_size + self.predict_size
        self.is_inference = is_inference
        self.batch_size = batch_size
    def __len__(self):
        if self.is_inference:
            return len(self.data)
        else:
            return self.data.shape[0] * (self.data.shape[1] - self.window_size - 3) ##//100  ## 소규모로 테스트를 원할 때 주석 지우고 사용

    def generate_example(self, idx):
        if self.is_inference:
            encode_info = self.data[idx, :4]
            window = self.data[idx, -self.train_size:]
            input_data = np.column_stack((np.tile(encode_info, (self.train_size, 1)), window))
            return (
                tf.convert_to_tensor(input_data, dtype=tf.float64),
                tf.convert_to_tensor(np.zeros((CFG['PREDICT_SIZE'],), dtype=np.float64), dtype=tf.float64),
            )
        else:
            row = idx // (self.data.shape[1] - self.window_size - 3)
            col = idx % (self.data.shape[1] - self.window_size - 3)
            encode_info = self.data[row, :4]
            sales_data = self.data[row, 4:]
            window = sales_data[col: col + self.window_size]
            input_data = np.column_stack((np.tile(encode_info, (self.train_size, 1)), window[:self.train_size]))
            target_data = window[self.train_size: self.train_size + self.predict_size]
            return (
                tf.convert_to_tensor(input_data, dtype=tf.float64),
                tf.convert_to_tensor(target_data, dtype=tf.float64),
            )

    def generator(self):
        for idx in range(len(self)):
            yield self.generate_example(idx)

In [59]:
val_size = int(len(train_data) * 0.2)
train_size = len(train_data) - val_size
val_data = CustomDataset(data=train_data, is_inference=False)
##train_data_small = train_data.head(train_size) ## 소규모로 테스트를 원할 때 주석 지우고 사용

train_dataset = tf.data.Dataset.from_generator(
    CustomDataset(train_data).generator,  ##  디폴트 : CustomDataset(train_data).generator / 소규모로 테스트를 원할 때 : CustomDataset(train_data_small).generator
    output_signature=(
        tf.TensorSpec(shape=(CFG['TRAIN_WINDOW_SIZE'], 5), dtype=tf.float64),
        tf.TensorSpec(shape=(CFG['PREDICT_SIZE']), dtype=tf.float64)
    )
).batch(CFG['BATCH_SIZE']).take(train_size)

val_dataset = tf.data.Dataset.from_generator(
    val_data.generator,
    output_signature=(
        tf.TensorSpec(shape=(CFG['TRAIN_WINDOW_SIZE'], 5), dtype=tf.float64),
        tf.TensorSpec(shape=(CFG['PREDICT_SIZE']), dtype=tf.float64)
    )
).batch(CFG['BATCH_SIZE'])

C:\Users\butterflywings02\AppData\Local\Temp\ipykernel_24932\3225025377.py:6: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  data.iloc[:, 4:] = np.where(is_equal[:, np.newaxis], 0, (data.iloc[:, 4:] - min_values[:, np.newaxis]) / (max_values - min_values)[:, np.newaxis])
C:\Users\butterflywings02\AppData\Local\Temp\ipykernel_24932\3225025377.py:6: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  data.iloc[:, 4:] = np.where(is_equal[:, np.newaxis], 0, (data.iloc[:, 4:] - min_values[:, np.newaxis]) / (max_values - min_values)[:, np.newaxis])


In [60]:
class BaseModel(tf.keras.Model):
    def __init__(self, input_size=5, hidden_size=512, output_size=CFG['PREDICT_SIZE']):
        super(BaseModel, self).__init__()
        self.hidden_size = hidden_size

        self.lstm1 = LSTM(hidden_size, return_sequences=True)
        self.dropout1 = Dropout(0.5)

        self.lstm2 = LSTM(hidden_size, return_sequences=True)
        self.dropout2 = Dropout(0.5)

        self.lstm3 = LSTM(hidden_size, return_sequences=False)

        self.fc = tf.keras.Sequential([
            Dense(hidden_size // 2),
            ReLU(),
            Dropout(0.5),
            Dense(output_size)
        ])

        self.actv = ReLU()

    def call(self, x):
        batch_size = tf.shape(x)[0]
        hidden = self.init_hidden(batch_size)

        x_lstm1 = x[:, :self.hidden_size, :]

        lstm_out1 = self.lstm1(x, initial_state=hidden)
        dropout_out1 = self.dropout1(lstm_out1)

        lstm_out2 = self.lstm2(dropout_out1)
        dropout_out2 = self.dropout2(lstm_out2)

        lstm_out3 = self.lstm3(dropout_out2)

        output = self.actv(self.fc(lstm_out3))

        return output

    def init_hidden(self, batch_size):
        return [tf.zeros((batch_size, self.hidden_size)), tf.zeros((batch_size, self.hidden_size))]

model = BaseModel()
optimizer = tf.keras.optimizers.Adam(learning_rate=CFG['LEARNING_RATE'])

In [61]:
def mae_loss(y_true, y_pred):
    y_true = tf.cast(y_true, dtype=tf.float64)
    y_pred = tf.cast(y_pred, dtype=tf.float64)
    return tf.reduce_mean(tf.abs(y_true - y_pred))

In [62]:
from tqdm import tqdm

In [ ]:
def train_step(inputs, targets):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = mae_loss(targets, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

epochs = CFG['EPOCHS']
train_losses = []
val_losses = []

for epoch in range(1, epochs + 1):
    train_loss = 0.0
    num_batches = 0
    for inputs, targets in tqdm(train_dataset, desc=f'Epoch {epoch} (Train)', leave=True):
        loss = train_step(inputs, targets)
        train_loss += loss
        num_batches += 1
    train_loss /= num_batches

    val_loss = 0.0
    num_val_batches = 0
    for inputs, targets in tqdm(val_dataset, desc=f'Epoch {epoch} (Validation)', leave=True):
        predictions = model(inputs, training=False)
        loss = mae_loss(targets, predictions)
        val_loss += loss
        num_val_batches += 1
    val_loss /= num_val_batches

    train_losses.append(train_loss)
    val_losses.append(val_loss)

    print(f'Epoch [{epoch}/{epochs}] Train Loss: {train_loss:.5f} Val Loss: {val_loss:.5f}')

Epoch 1 (Train): 1525it [1:36:15,  3.93s/it]

In [37]:
test_data = pd.read_csv('./Desktop/open/train.csv').drop(columns=['ID', '제품','대분류', '중분류', '소분류', '브랜드'])


test_dataset = CustomDataset(data=test_data, is_inference=True)

In [39]:
test_tf_dataset = tf.data.Dataset.from_generator(
    test_dataset.generator,
    output_signature=(
        tf.TensorSpec(shape=(CFG['TRAIN_WINDOW_SIZE'], 5), dtype=tf.float64),
        tf.TensorSpec(shape=(CFG['PREDICT_SIZE']), dtype=tf.float64)
    )
).batch(CFG['BATCH_SIZE'])

In [42]:
def inference_tf(model, test_tf_dataset):
    predictions = []

    for X_batch in tqdm(test_tf_dataset):
        input_data = X_batch[0]
        output_data = X_batch[1]
        output = model(input_data, training=False)

        output = output.numpy()

        predictions.extend(output)

    return np.array(predictions)

In [43]:
pred = inference_tf(model, test_tf_dataset)

4it [00:08,  2.10s/it]


In [44]:
for idx in range(len(pred)):
    pred[idx, :] = pred[idx, :] * (scale_max_dict[idx] - scale_min_dict[idx]) + scale_min_dict[idx]

pred = np.round(pred).astype(int)

In [45]:
pred.shape

(15890, 21)

In [46]:
submit = pd.read_csv('./Desktop/open/sample_submission.csv')
submit.head()

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
submit.iloc[:,1:] = pred

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15885,15885,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15886,15886,0,0,2,2,0,1,0,1,1,...,0,0,0,0,0,1,0,0,0,0
15887,15887,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15888,15888,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
submit.to_csv('./Desktop/open/sample_submission3.csv', index=False)